<a href="https://colab.research.google.com/github/informatics-isi-edu/eye-ai-exec/blob/main/notebooks/VGG19_Diagnosis_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VGG19 Hyper Parameter Tuning

This notebook is used to train VGG19 model for glacoma diagnosis.

In [1]:
# import sys
# IN_COLAB = 'google.colab' in sys.modules

# if IN_COLAB:
#     !pip install deriva
#     !pip install bdbag
#     !pip install --upgrade --force pydantic
#     !pip install git+https://github.com/informatics-isi-edu/deriva-ml git+https://github.com/informatics-isi-edu/eye-ai-ml

In [1]:
repo_dir = "Repos"   # Set this to be where your github repos are located.
%load_ext autoreload
%autoreload 2

# Update the load path so python can find modules for the model
import sys
from pathlib import Path
sys.path.insert(0, str(Path.home() / repo_dir / "eye-ai-ml"))

In [2]:
# Prerequisites

import json
import os
from eye_ai.eye_ai import EyeAI
import pandas as pd
from pathlib import Path, PurePath
import logging
# import torch

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

In [3]:

from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
catalog_id = "eye-ai" #@param
host = 'www.eye-ai.org'


gnl = GlobusNativeLogin(host=host)
if gnl.is_logged_in([host]):
    print("You are already logged in.")
else:
    gnl.login([host], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")

2024-04-30 20:04:52,078 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-04-30 20:04:52,079 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>


You are already logged in.


Connect to Eye-AI catalog.  Configure to store data local cache and working directories.  Initialize Eye-AI for pending execution based on the provided configuration file.

In [4]:
# Variables to configure the rest of the notebook.

cache_dir = '/data'        # Directory in which to cache materialized BDBags for datasets
working_dir = '/data'    # Directory in which to place output files for later upload.

configuration_rid="2-A7D4" # rid
# Change the confi_file with bag_url=["minid: train", "minid: Valid", "minid: test"]


In [5]:
EA = EyeAI(hostname = host, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)

2024-04-30 20:04:55,449 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-04-30 20:04:55,450 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>


In [6]:
# @title Initiate an Execution
configuration_records = EA.execution_init(configuration_rid=configuration_rid)
configuration_records.model_dump()

2024-04-30 20:04:56,117 - INFO - File [/data/munimadu/EyeAI_working/Execution_Metadata/Execution_Config-vgg19_catalog_model_training_sreenidhi_apr_27_2024.json] transfer successful. 0.74 KB transferred. Elapsed time: 0:00:00.000071.
2024-04-30 20:04:56,117 - INFO - Verifying MD5 checksum for downloaded file [/data/munimadu/EyeAI_working/Execution_Metadata/Execution_Config-vgg19_catalog_model_training_sreenidhi_apr_27_2024.json]
2024-04-30 20:04:56,144 - INFO - Configuration validation successful!


{'caching_dir': PosixPath('/data'),
 'working_dir': PosixPath('/data/munimadu/EyeAI_working'),
 'vocabs': {'Workflow_Type': [{'name': 'VGG19_Catalog_Model_training',
    'rid': '2-A7D8'}],
  'Execution_Asset_Type': [{'name': 'VGG19_Catalog_Model', 'rid': '2-A7DA'}]},
 'execution_rid': '2-A7DJ',
 'workflow_rid': '2-A7CR',
 'bag_paths': [PosixPath('/data/2-277G_6aa1a6861eee5a79bce4bf071065355f95a066c2a1ff326089d43048a7e0f185/Dataset_2-277G'),
  PosixPath('/data/2-277J_81c873a311aa6a67cf2eef44bd9056cb19181b299a6e44327ea3553616f18725/Dataset_2-277J'),
  PosixPath('/data/2-277M_8c4b855c2752e098580a5bb0d1b63a8cedde4462805fe74cddc912a72fb39963/Dataset_2-277M')],
 'assets_paths': [],
 'configuration_path': PosixPath('/data/munimadu/EyeAI_working/Execution_Metadata/Execution_Config-vgg19_catalog_model_training_sreenidhi_apr_27_2024.json')}

In [7]:
configuration_records

ConfigurationRecord(caching_dir=PosixPath('/data'), working_dir=PosixPath('/data/munimadu/EyeAI_working'), vocabs={'Workflow_Type': [Term(name='VGG19_Catalog_Model_training', rid='2-A7D8')], 'Execution_Asset_Type': [Term(name='VGG19_Catalog_Model', rid='2-A7DA')]}, execution_rid='2-A7DJ', workflow_rid='2-A7CR', bag_paths=[PosixPath('/data/2-277G_6aa1a6861eee5a79bce4bf071065355f95a066c2a1ff326089d43048a7e0f185/Dataset_2-277G'), PosixPath('/data/2-277J_81c873a311aa6a67cf2eef44bd9056cb19181b299a6e44327ea3553616f18725/Dataset_2-277J'), PosixPath('/data/2-277M_8c4b855c2752e098580a5bb0d1b63a8cedde4462805fe74cddc912a72fb39963/Dataset_2-277M')], assets_paths=[], configuration_path=PosixPath('/data/munimadu/EyeAI_working/Execution_Metadata/Execution_Config-vgg19_catalog_model_training_sreenidhi_apr_27_2024.json'))

In [9]:
# @title Data Preprocessing (Filtering Image.csv for just Field_2 Images)
train_dir = configuration_records.bag_paths[0] # path to the raw train dataset
validation_dir = configuration_records.bag_paths[1]
test_dir = configuration_records.bag_paths[2]

# train_cropped_image_path = path to training cropped images
train_cropped_image_path, train_cropped_csv = EA.create_cropped_images(str(train_dir),
                                                                     output_dir = str(EA.working_dir) +'/train',
                                                                       crop_to_eye=True)
validation_cropped_image_path, validation_cropped_csv = EA.create_cropped_images(str(validation_dir),
                                                                                 output_dir = str(EA.working_dir) +'/valid',
                                                                                 crop_to_eye=True)
test_cropped_image_path, test_cropped_csv = EA.create_cropped_images(str(test_dir),
                                                                     output_dir = str(EA.working_dir) +'/test',
                                                                     crop_to_eye=True)


In [10]:
output_path = str(EA.working_dir) + configuration_records.vocabs['Execution_Asset_Type'][0].name
os.mkdir(output_path)

In [11]:
# @title Execute Training algorithm
from eye_ai.models.vgg19_diagnosis_train import main
with EA.execution(execution_rid=configuration_records.execution_rid) as exec:
  main(train_path=train_cropped_image_path,
       valid_path=validation_cropped_image_path, 
       test_path=test_cropped_image_path, 
       output_path = output_path                    )
                    


2024-04-30 20:16:14.595738: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-30 20:16:14.595779: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-30 20:16:14.597810: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-30 20:16:14.609562: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-30 20:16:16.390480: W tensorflow/compiler/tf2

Found 9065 images belonging to 2 classes.
Found 3033 images belonging to 2 classes.
Found 1094 images belonging to 2 classes.
train_generator.class_indices :  {'2SKC_No_Glaucoma': 0, '2SKA_Suspected_Glaucoma': 1}
validation_generator.class_indices :  {'2SKC_No_Glaucoma': 0, '2SKA_Suspected_Glaucoma': 1}
test_generator.class_indices :  {'2SKC_No_Glaucoma': 0, '2SKA_Suspected_Glaucoma': 1}


2024-04-30 20:16:18.472667: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-30 20:16:18.603301: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-30 20:16:18.607034: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Epoch 1/100


2024-04-30 20:16:23.060354: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2024-04-30 20:16:27.954536: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f332cd70d70 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-04-30 20:16:27.954576: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A10G, Compute Capability 8.6
2024-04-30 20:16:27.960309: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1714533388.051344   31596 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


284/284 [==============================] - 79s 233ms/step - loss: 0.7186 - roc_auc_score: 0.6969 - f1_score_normal: 0.5268 - accuracy_score: 0.6684 - val_loss: 0.5629 - val_roc_auc_score: 0.8241 - val_f1_score_normal: 0.6870 - val_accuracy_score: 0.7102
Epoch 2/100
284/284 [==============================] - 68s 235ms/step - loss: 0.5722 - roc_auc_score: 0.8033 - f1_score_normal: 0.6597 - accuracy_score: 0.7356 - val_loss: 0.5005 - val_roc_auc_score: 0.8520 - val_f1_score_normal: 0.7136 - val_accuracy_score: 0.7692
Epoch 3/100
284/284 [==============================] - 65s 227ms/step - loss: 0.5379 - roc_auc_score: 0.8298 - f1_score_normal: 0.6979 - accuracy_score: 0.7609 - val_loss: 0.4608 - val_roc_auc_score: 0.8534 - val_f1_score_normal: 0.6894 - val_accuracy_score: 0.7709
Epoch 4/100
284/284 [==============================] - 70s 241ms/step - loss: 0.5044 - roc_auc_score: 0.8477 - f1_score_normal: 0.7160 - accuracy_score: 0.7711 - val_loss: 0.7082 - val_roc_auc_score: 0.8710 - val_f

2024-04-30 20:49:25,179 - INFO - Test results - [0.46174943447113037, 0.8891581296920776, 0.8132581114768982, 0.809872031211853]
/home/munimadu/.conda/envs/my-tensorflow-conda/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model Eval results: [0.46174943447113037, 0.8891581296920776, 0.8132581114768982, 0.809872031211853]


2024-04-30 20:49:25,424 - INFO - VGG19_Catalog_LAC_DHS_Cropped_Data_Trained_model.h5 Model trained, Model and training history are saved successfully.


In [15]:
# @title Save Execution Assets (model) and Metadata
uploaded_assets = EA.execution_upload(configuration_records.execution_rid, False)

2024-04-30 20:49:37,178 - INFO - Initializing uploader: GenericUploader v1.7.0 [Python 3.10.13, Linux-5.10.210-201.852.amzn2.x86_64-x86_64-with-glibc2.26]
2024-04-30 20:49:37,179 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-04-30 20:49:37,180 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>
2024-04-30 20:49:37,215 - INFO - Checking for updated configuration...
2024-04-30 20:49:37,371 - INFO - Updated configuration found.
2024-04-30 20:49:37,372 - INFO - Scanning files in directory [/data/munimadu/EyeAI_working/Execution_Assets/VGG19_Catalog_Model]...
2024-04-30 20:49:37,376 - INFO - Including file: [/data/munimadu/EyeAI_working/Execution_Assets/VGG19_Catalog_Model/VGG19_Catalog_LAC_DHS_Cropped_Data_Trained_model.h5].
2024-04-30 20:49:37,377 - INFO - Including file: [/data/munimadu/E

# 